In [1]:
# Imports
from IOHexperimenter import IOH_function, IOH_logger, IOHexperimenter
import input_reader
import parameters
import datetime
from functools import partial

In [2]:
# Create a dictionary that contains algorithms and splitpoints / targets
data = input_reader.create_dict()

In [3]:
class Switch:
    """ Class to implement the switch between algorithms.

    Parameters:
    ---------------
    func : object, callable
        The function to be optimized.

    budget : int
        The budget that is available for function evaluations.

    data : dict
        A dictionary with tau as key, algorithm as value


    Attibutes:
    ---------------
    None

    """

    def __init__(self, budget_factor = 1e4):
        self.budget_factor = budget_factor

    def __call__(self, func, switch_dict, budget = None):
        """ Runs the switch routine. 
        
        Returns:
        -----------
        x_opt : array-type
            The best found solution.

        f_opt: float
            The fitness value for the best found solution.
        """
        assert isinstance(func, IOH_function), "The provided function should be of the IOH_function type"
        assert isinstance(switch_dict, dict), "The provided switching dict needs to be a dictionary"
        #There (later) should be a check here to get whether the tau's are in decreasing order (and either raise an error or fix it)
        
        if budget is None:
            budget = func.number_of_variables * self.budget_factor
        
        # Initialize parameters
        params = parameters.init_params(budget)

        # Iterate through dictionary
        for tau, alg in switch_dict.items():
            # Would be good to check here isinstance(alg, algorithm)
            algorithm = alg(func)
            # Set algorithm parameters based on parameters object
            algorithm.set_params(params)

            # Define stopping criteria
            def stopping_criteria():
                return (func.best_so_far_precision) <= tau or (
                    func.evaluations > budget)

            algorithm.set_stopping_criteria(stopping_criteria)

            # Run algorithm and extract parameters
            algorithm.run()
            params = algorithm.get_params()

        # Return best point and fitness value found by last run algorithm
        return algorithm.x_opt, algorithm.f_opt

In [12]:
# Experiment routine
path = ""
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(
    now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name = path + "switch-routine/experiment" + experiment_ID
dimensions = [3]
instances = [1, 2, 3, 4, 5]
func_ids = [20]
exp_budget = 10000
runs = 5

switch_routine = Switch()

# for func_id in func_ids:
#     for dim in dimensions:
#         for inst in instances:
#             f = IOH_function(func_id, dim, inst, suite='BBOB')
#             log = IOH_logger(experiment_name, "run",
#                              "switch-routine", "switch-routine")
#             f.add_logger(log)
#             for run in range(0, runs):
#                 if run > 0:
#                     f.reset()
#                 switch(f, data, exp_budget)
#                 print(f' run {run} dim{dim} f{func_id} inst{inst} complete')
#             f.clear_logger()
            


In [ ]:
exp = IOHexperimenter()
exp.initialize_BBOB(func_ids, instances, dimensions, runs, 0)
exp.set_logger_location(experiment_name)
# exp.set_parallel(True, "pebble") Optional for running parallel, several libraries are available, but might still be unstable for some?
switch_routine = partial(switch_routine, switch_dict = data, budget = exp_budget)
exp([switch_routine])

Running pebble pool with 5 threads
202020   15951567157720    0.52501689476319480.0172168684114478760.487844540213164371550

